In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32636417
paper_name = 'fu_hoepfner_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheets = ['Raw Data HIP HOP Exp.1 Fig 2ab','Raw Data HIP HOP Exp.2 Fig 2ab']

In [6]:
original_data1_list = []
original_data2_list = []

for s in sheets:
    original_data = pd.read_excel('raw_data/SourceData_TableFormats.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data.head()
    original_data['orf'] = original_data['SYSTEMATIC_NAME'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = pd.to_numeric(original_data['SCORE'], errors='coerce')
    original_data.set_index('orf', inplace=True)
    
    original_data1 = original_data.loc[original_data['EXPERIMENT_TYPE']=='HOP'].copy()
    original_data2 = original_data.loc[original_data['EXPERIMENT_TYPE']=='HIP'].copy()
    
    original_data1 = original_data1[['data']].copy()
    original_data2 = original_data2[['data']].copy()
    
    original_data1 = original_data1.groupby(original_data1.index).mean()
    original_data2 = original_data2.groupby(original_data2.index).mean()
    
    print(original_data1.shape)
    print(original_data2.shape)
    
    original_data1_list.append(original_data1)
    original_data2_list.append(original_data2)

Original data dimensions: 14634 x 8
Empty DataFrame
Columns: [COMPOUND, EXPERIMENT_TYPE, GENE_NAME, SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(4482, 1)
(5701, 1)
Original data dimensions: 10130 x 8
Empty DataFrame
Columns: [COMPOUND, EXPERIMENT_TYPE, GENE_NAME, SCORE, SCORE_TYPE, SYSTEMATIC_NAME, VIABILITY, Z_SCORE, orf]
Index: []
(4431, 1)
(5683, 1)


In [7]:
original_data1 = pd.concat(original_data1_list, axis=0)
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data1.shape

(4485, 1)

In [8]:
original_data2 = pd.concat(original_data2_list, axis=0)
original_data2 = original_data2.groupby(original_data2.index).mean()
original_data2.shape

(5716, 1)

In [9]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [10]:
original_data.shape

(5814, 2)

# Prepare the final dataset

In [11]:
data = original_data.copy()

In [12]:
dataset_ids = [21880, 21879]
datasets = datasets.reindex(index=dataset_ids)

In [13]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [14]:
data.head()

dataset_id,21880,21879
data_type,value,value
orf,,
YAL001C,0.111621,0.203423
YAL002W,-0.202490,3.752144
YAL003W,NaN,-0.056619
YAL004W,0.198531,-2.075475
YAL005C,0.118330,1.490524


## Subset to the genes currently in SGD

In [15]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 10


In [16]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21880,21879
,data_type,value,value
gene_id,orf,,
1,YAL001C,0.111621,0.203423
2,YAL002W,-0.202490,3.752144
3,YAL003W,NaN,-0.056619
1863,YAL004W,0.198531,-2.075475
4,YAL005C,0.118330,1.490524


# Normalize

In [17]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [18]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [19]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21880,21879,21880,21879
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1,YAL001C,0.111621,0.203423,0.122733,0.083099
2,YAL002W,-0.202490,3.752144,-0.011415,1.664446
3,YAL003W,NaN,-0.056619,NaN,-0.032778
1863,YAL004W,0.198531,-2.075475,0.159849,-0.932401
4,YAL005C,0.118330,1.490524,0.125598,0.656645


# Print out

In [20]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [21]:
from IO.save_data_to_db3 import *

In [22]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 32636417...
Inserting the new data...


100%|██████████| 2/2 [00:16<00:00,  8.22s/it]

Updating the data_modified_on field...
